<a href="https://colab.research.google.com/github/LizCarter492/EnvDatSci/blob/main/CodeSprints/VectorData101_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Vector Data in Python w/ Geopandas
You will use the geopandas library to work with vector data in Python. Geopandas is built on top of the Python Pandas library. It stores spatial data in a tabular, dataframe format.

This notebook will take you through [Lesson 3 in Introductory Earth Data Science](https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-spatial-data/use-vector-data/) with some additional exercises from the [Geopandas project page](https://geopandas.org/en/v0.8.2/)


# Sync your colab environment with your Google Drive and GitHub

In [ ]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# The Code Sprint

In [ ]:
!pip install geopandas

In [ ]:
!pip install earthpy
!pip install rtree
!pip install pygeos
!pip install mapclassify>=2.4.0

In [4]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import earthpy as et


To begin, set your working directory to earth-analytics and then download a single shapefile.

You will start with working with the Natural Earth country boundary lines layer: https://www.naturalearthdata.com/downloads/

Note that below you are using EarthPy to download a dataset from naturalearthdata.com (via Amazon Web Services).

EarthPy automatically creates the earth-analytics directory for you when you use it, but by default makes this directory in your home directory. We're doing this here because you guys are on PCs.

You set the working directory after you download the data as a precaution to ensure that the earth-analytics directory already exists on your computer. This is not a standard order of operations, as we are not using our SU H drive, but we are demonstrating it here to ensure the notebook runs on all computers!

In [ ]:
# Download a political boundaries shapefile and set your working directory
et.data.get_data(
    url='https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_boundary_lines_land.zip')

# Set working directory - earthpy creates earth-analytics for you in your home dir
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

### geopandas
Next, you open the data using geopandas.

***geopandas*** takes all of the data science magic from the pandas library and makes it compatible with shapfiles.

Learn more here: https://geopandas.org/

You can view the first 5 rows of the data using .head() in the same way you used .head() for Pandas dataframes.¶

In [ ]:
os.getcwd()

We're going to download a second dataset, called coastlines:

In [ ]:
#set URL for the dataset
coastlines_url = "https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_coastline.zip"

#download the URL into your earth-analytics folder
et.data.get_data(url=coastlines_url)

#name the filepath relative to your working directory: note this is operating system agnostic
coastlines_path = os.path.join("data", "earthpy-downloads",
                               "ne_50m_coastline",
                               "ne_50m_coastline.shp")

#use geopandas as gpd to "read_file"
coastlines = gpd.read_file(coastlines_path)

#prints "head" just like pandas!
coastlines.head()

## GeoPandas creates GeoDataFrames
The structure of a Geopandas GeoDataFrame is very similar to a Pandas dataframe. This means that all the awesome data wrangling functionality of pandas dataframes can be applied to our spatial vector data as well.

### A few differences include:
The GeoDataFrame contains a geometry column which stores spatial information. The geometry column in your GeoDataFrame stores the boundary information (the lines that make up each shape in your data). This allows you to plot points, lines or polygons.

The GeoDataFrame stores spatial attributes such as coordinate reference systems and spatial extents.

Similar to Pandas, you can plot the data using .plot()

In [ ]:
# Plot the data
f, ax1 = plt.subplots(figsize=(12, 6))
coastlines.plot(ax=ax1)

# Add a title to your plot
ax1.set(title="Global Coastline Boundaries")
plt.show()

### Check the Spatial Vector Data Type
You can look at the data to figure out what type of data are stored in the shapefile (points, line or polygons). However, you can also get that information by calling .geom_type

In [ ]:
# Is the geometry type point, line or polygon?
coastlines.geom_type

Also similar to Pandas, you can view descriptive information about the GeoDataFrame using .info(). This includes the number of columns, rows and the header name and type of each column.¶

In [ ]:
coastlines.info()

### Open point data
Next, you will accesss another online shapefile using Geopandas.

In [52]:
# Open a second layer
et.data.get_data(
    url='https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_populated_places_simple.zip')

# Create a path to the populated places shapefile
populated_places_path = os.path.join("data", "earthpy-downloads",
                                     "ne_50m_populated_places_simple",
                                     "ne_50m_populated_places_simple.shp")

#read in a new geopandas data frame called "cities"
cities = gpd.read_file(populated_places_path)

### TASK 1: Is cities a point, line, or polygon file? Type a command to find out and interpret the answer.

In [12]:
# Task 1 answer here

The attributes for a shapefile imported into a GeoDataFrame can be viewed in the GeoDataFrame itself.

In [ ]:
# View first 5 rows of GeoDataFrame
cities.head()


# Experiment! How would you view the first 10 rows of a GeoDataFrame?

Just like with Pandas DataFrames, standard arguments can be used to calculate summary statistics on your GeoPandas object:

In [ ]:
#Let's see what our most populous city is
#print out the column with maximum population
print(cities.pop_max)
print(cities.iloc[cities.pop_max.idxmax()])

print("The world's largest city  by population is " + cities.name.iloc[cities.pop_max.idxmax()] +
      " with " + str(cities.pop_max.max()) +
      " people!")

## Geopandas = geographic pandas data frames
The handy thing about working with vector data in geopandas is that you have all of the functionality of pandas at your fingertips. This means that it is easy to do things like:
1. Subset your shapefile based on certain crtiera
2. Merge your shapefile with other datasets
3. Process fields (columns) in your shapefile using python commands.

For example, let's say we are conducting a study on megacities. We can easily identify our megacities using the pop_max field, and use that to create a new geopandas DataFrame of only megacities (cities with a population greater than 10 million.

In [ ]:
megacities = cities[cities.pop_max >= 10000000]
print(megacities)
type(megacities)

### Creating Maps Using Multiple Shapefiles
You can create maps using multiple shapefiles with Geopandas in a similar way that you may do so using a graphical user interface (GUI) tool like ArcGIS or QGIS (open source alternative to ArcGIS). To do this you will need to open a second spatial file. Below you will use the Natural Earth populated places shapefile to add additional layers to your map.

To plot two datasets together, you will first create a Matplotlib figure object. Notice in the example below that you define the figure ax1 in the first line. You then tell GeoPandas to plot the data on that particular figure using the parameter ax=

The code looks like this:

In [ ]:
#Experiment!
#What happens if you change FigSize numbers?
#What happens if you "comment out" (put a # in front of) plt.show()?
#What happens if you change the color?

f, ax1 = plt.subplots(figsize=(10, 6))
coastlines.plot(ax=ax1,
               color = "black")
plt.show()

To add another layer to your map, you can add a second .plot() call and specify the ax= to be ax1 again. This tells Python to layer the two datasets in the same figure. Identify the lines of code that differ between the cell above and below. What do these lines of code do?

In [ ]:
# Create a map or plot with two data layers

#Experiment! What happens if you remove ax=ax1 from coastlines.plot() and cities.plot()?

f, ax1 = plt.subplots(figsize=(10, 6))
coastlines.plot(ax=ax1,
                color="black")
cities.plot(ax=ax1)

# Add a title
ax1.set(title="Map of Cities and Global Lines")
plt.show()

Learn more about custom plotting in Python:

https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-vector-plots/python-customize-map-legends-geopandas/

https://geopandas.org/gallery/plotting_with_geoplot.html

### TASK 2
Using the resources above, create a map that contains:
1. Coastal lines (black)
2. Cities, excluding megacities (points), with a color scale for pop_max
3. Megacities (large black points)
3. A legend

In [18]:
# Format and plot your results from task 2
# hint:
?cities.plot

### Geoprocessing Vector Data Geoprocessing in Python: Clip Data
Sometimes you have spatial data for a larger area than you need to process. For example you may be working on a project for your state or country. But perhaps you have data for the entire globe.

You can clip the data spatially to another boundary to make it smaller. Once the data are clipped, your processing operations will be faster. It will also make creating maps of your study area easier and cleaner.

In this workflow, you'll subset your cities data to only look at counties in the United States. First, we'll import a dataset of global political boundaries.

In [ ]:
country_data_url = "https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_countries.zip"
et.data.get_data(url=country_data_url)

# Create a path to the countries shapefile
countries_path = os.path.join("data", "earthpy-downloads",
                              "ne_50m_admin_0_countries",
                              "ne_50m_admin_0_countries.shp")

# Read in the countries shapefile as GeoPandas dataframe
countries = gpd.read_file(countries_path)

# View attribute table:
countries.head()

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
countries.plot(ax=ax)
plt.show()

### Next, we'll use built-in pandas funcitonality to subset the shapefile to just the US boundary

In [ ]:
# Subset the countries data to just a single country
united_states_boundary = countries.loc[countries['SOVEREIGNT']
                                       == 'United States of America']

# Notice in the plot below, that only the boundary for the USA is in the new variable
f, ax = plt.subplots(figsize=(10, 6))
united_states_boundary.plot(ax=ax)
plt.show()

### Now, we'll subset the cities layer to include only records which overlap in space with the united_states_boundary layer (aka are in the United States)

In [ ]:
# Clip the cities data to the USA boundary
# Note -- this operation may take some time to run - be patient
cities_in_usa = gpd.clip(cities, united_states_boundary)

# Plot your final clipped data
f, ax = plt.subplots()
cities_in_usa.plot(ax=ax)
ax.set(title="Cities clipped to the USA Boundary")
plt.show()

## TASK 3: How many cities are in the United States?

In [23]:
# Task 3 code here

## TASK 4: What is the largest city in the United States? What is the population of that city?


In [24]:
#Task 4 code here


## Merging vector data in geopandas
One of our primary goals using geographic information systems (GIS) software in Environmental Data Science is to collocate information from different sources into a single analaysis-ready dataset. When we're working with vector data, this often means adding new columns to a geopandas dataframe containing values from another dataset.

There are two ways to combine datasets in geopandas – attribute joins and spatial joins. [From Geopandas.org](https://geopandas.org/en/v0.8.2/mergingdata.html)

*   **Attribute joins:** a GeoSeries or GeoDataFrame is combined with a regular pandas Series or DataFrame based on a common variable. This is analogous to normal merging or joining in pandas.

*   **Spatial joins:** a Spatial Join, observations from two GeoSeries or GeoDataFrames are combined based on their spatial relationship to one another.Indented block

First, we'll bring in another dataset of countries:


In [ ]:
# read in country shapefile from the geopandas datasets collection
world = et.data.get_data(url='https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip')
world_path = os.path.join("data", "earthpy-downloads",
                                     "ne_110m_admin_0_countries",
                                     "ne_110m_admin_0_countries.shp")

#read in a new geopandas data frame called "cities"
world = gpd.read_file(world_path)
print(world)

Let's say we want to add the "continent" field (column) in the world geodataframe to the cities dataframe. Using an attribute join, we first find two columns that contain the same values:

In [ ]:
world.head()

In [ ]:
print(sorted(world.SOV_A3.unique()))
print(sorted(cities.sov_a3.unique()))
print("There are " + str(len(world.SOV_A3.unique()))+ " unique country codes in the world dataset")
print("There are " + str(len(cities.sov_a3.unique()))+ " unique country codes in the city dataset")

We do not have continent data for all of the countries represented in the city database, but we have some!

In [56]:
# First, we'll make a new pandas dataframe containing only the  iso_a3, and continent information:
continents = world[['SOV_A3', 'CONTINENT']]

In [ ]:
# Then, we'll merge this data to the cities dataset:
cities = cities.merge(continents, left_on='sov_a3', right_on='SOV_A3')

# Is cities still a geopandas dataframe?
type(cities)

#On what continents do we have cities?
cities.CONTINENT.unique()

A spatial join involves fusing two datasets based on the euclidian distances calculated between feature locations.


Sjoin Arguments

`sjoin()` has two core arguments: how and op.

**op**

The op argument specifies how geopandas decides whether or not to join the attributes of one object to another. There are three different join options as follows:

    intersects: The attributes will be joined if the boundary and interior of the object intersect in any way with the boundary and/or interior of the other object.

    within: The attributes will be joined if the object’s boundary and interior intersect only with the interior of the other object (not its boundary or exterior).

    contains: The attributes will be joined if the object’s interior contains the boundary and interior of the other object and their boundaries do not touch at all.

You can read more about each join type in the Shapely documentation.

**how**

The how argument specifies the type of join that will occur and which geometry is retained in the resultant geodataframe. It accepts the following options:

    left: use the index from the first (or left_df) geodataframe that you provide to sjoin; retain only the left_df geometry column

    right: use index from second (or right_df); retain only the right_df geometry column

    inner: use intersection of index values from both geodataframes; retain only the left_df geometry column

Note more complicated spatial relationships can be studied by combining geometric operations with spatial join. To find all polygons within a given distance of a point, for example, one can first use the buffer method to expand each point into a circle of appropriate radius, then intersect those buffered circles with the polygons in question.


In [ ]:
world.head()

In [ ]:
# Create a countries shapefile with only one attribute (column), called 'SOVEREIGNT':
countries = world[['geometry', 'SOVEREIGNT']]
# Rename "SOVERIGNT" to "name"
countries = countries.rename(columns={"geometry": "geometry", "SOVEREIGNT": "name"})
countries.head()


In [66]:
# Recall all the attributes of cities:
cities.head()

,scalerank,natscale,labelrank,featurecla,name,namepar,namealt,nameascii,adm0cap,capalt,...,pop_other,rank_max,rank_min,meganame,ls_name,min_zoom,ne_id,geometry,SOV_A3,CONTINENT
0,10,1,5,Admin-1 region capital,Bombo,None,None,Bombo,0,0,...,0.0,8,7,None,None,7.0,1159113923,POINT (32.5333 0.5833),UGA,Africa
1,10,1,5,Admin-1 region capital,Fort Portal,None,None,Fort Portal,0,0,...,0.0,7,7,None,None,7.0,1159113959,POINT (30.275 0.671),UGA,Africa
2,10,1,3,Admin-1 region capital,Potenza,None,None,Potenza,0,0,...,0.0,8,8,None,None,7.0,1159117259,POINT (15.799 40.642),ITA,Europe
3,10,1,3,Admin-1 region capital,Campobasso,None,None,Campobasso,0,0,...,0.0,8,8,None,None,7.0,1159117283,POINT (14.656 41.563),ITA,Europe
4,10,1,3,Admin-1 region capital,Aosta,None,None,Aosta,0,0,...,0.0,7,7,None,None,7.0,1159117361,POINT (7.315 45.737),ITA,Europe


In [ ]:
# Check and see if the merge worked
cities_with_country = gpd.sjoin(cities, countries, how="inner", predicate='intersects')
cities_with_country.head()

## TASK 5:
In this markdown cell in your own words, explain how the spatial join worked. How does this differ from the attribute join?

## Nested features
Often, vector spatial features will be nested within each other. For example, in the world dataset, there are countries which are nested within continents.

If you wanted to extract just the continent outlines, you would need to erase the country boundaries within each continent. We can use the **dissolve** function to do this.

In a non-spatial setting, when all we need are summary statistics of the data, we aggregate our data using the `groupby` function. But for spatial data, we sometimes alsoIn a non-spatial setting, when all we need are summary statistics of the data, we aggregate our data using the groupby function. But for spatial data, we sometimes also need to aggregate geometric features. In the geopandas library, we can aggregate geometric features using the dissolve function.

dissolve can be thought of as doing three things: (a) it dissolves all the geometries within a given group together into a single geometric feature (using the unary_union method), and (b) it aggregates all the rows of data in a group using groupby.aggregate(), and (c) it combines those two results.

```
# This is formatted as code
```

 need to aggregate geometric features. In the geopandas library, we can aggregate geometric features using the `dissolve` function.

`dissolve` can be thought of as doing three things: (a) it dissolves all the geometries within a given group together into a single geometric feature (using the unary_union method), and (b) it aggregates all the rows of data in a group using `groupby.aggregate()`, and (c) it combines those two results.

In [ ]:
world = world[['CONTINENT', 'geometry']]

continents = world.dissolve(by='CONTINENT')

continents.plot();

continents.head()

If we are interested in aggregate populations, however, we can pass different functions to the dissolve method to aggregate populations using the `aggfunc = ` argument:

In [ ]:
# Re-read in the world dataset
world = et.data.get_data(url='https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip')
world_path = os.path.join("data", "earthpy-downloads",
                                     "ne_110m_admin_0_countries",
                                     "ne_110m_admin_0_countries.shp")

#read in a new geopandas data frame called "cities"
world = gpd.read_file(world_path)

# Select only the attributes we're interested in:
world = world[['CONTINENT', 'geometry', 'POP_EST']]

# Dissolve into country aggregates, collecting the sum of other attributes
continents = world.dissolve(by='CONTINENT', aggfunc='sum')

continents.plot(column = 'POP_EST', scheme='quantiles', cmap='YlOrRd');
continents.head()

## TASK 6-8: putting it together
Use your skills with `merge`, `dissolve`, and pandas math to do the following:
6. What is the total urban population on each continent? Show your results as a chloropleth map and a table.
7. What fraction of the total population lives in cities in each continent? Show your results as a chloropleth map and a table.
8. What is the most urban continent on earth (largest share of the population lives in cities)?

In [ ]:
 # Task 6 code here:

In [ ]:
# Task 6 code here:

In [ ]:
# Task 8 code here: